## Import of Bruker OPUS files

[Bruker OPUS](https://www.bruker.com/en/products-and-solutions/infrared-and-raman/opus-spectroscopy-software.html)
files have also a proprietary file format. The Opus reader (`read_opus()` ) of spectrochempy is essentially a wrapper of the python module
[brukeropus](https://github.com/joshduran/brukeropus) developed by Josh Duran.
The use of `read_opus()` is similar to that of  `read_omnic()`.

### Basic loading of OPUS file

Individual Opus files can be opened and loaded as a new NDDataset using:

In [1]:
import spectrochempy as scp

In [2]:
Z = scp.read_opus("irdata/OPUS/test.0002")
Z

Running on GitHub Actions
MPL Configuration directory: /home/runner/.config/matplotlib
Stylelib directory: /home/runner/.config/matplotlib/stylelib


NDDataset: [float64] a.u. (shape: (y:1, x:2567))

<div class='alert alert-info'>
<b>Note:</b><br/>
In the previous use of <strong>read_opus()</strong>, we have assumed that the file is in <strong>datadir</strong> folder (see the <a href="../import.html">import tutorial</a> or more details on this). <br/>If this not the case, you should use an absolute path or a relative path to the current notebook folder.
</div>

For multifile loading, one can use:

In [3]:
Z1 = scp.read_opus("test.0000", "test.0001", "test.0002", directory="irdata/OPUS")
Z1

NDDataset: [float64] a.u. (shape: (y:3, x:2567))

<div class='alert alert-info'>
<b>Note:</b><br/>
    By default all files in the given directory are merged as a single dataset if they are compatibles (*i.e.*, same shape, same type of experiment...).<br/>
    If they cannot be merged due to imcompatible shape or type, separate datasets are returned with dataset merged in different groups.

For instance in the following, two dataset will be returned:

In [4]:
LZ1 = scp.read_opus(
    "test.0000", "test.0001", "test.0002", "background.0", directory="irdata/OPUS"
)
LZ1

[NDDataset: [float64] a.u. (shape: (y:3, x:2567)),
 NDDataset: [float64] unitless (shape: (y:1, x:4096))]

Multifile loading can also be achieved by only specifying the directory to read, if all files in a directory must be read:

In [5]:
LZ = scp.read_opus("irdata/OPUS")
LZ

[NDDataset: [float64] a.u. (shape: (y:4, x:2567)),
 NDDataset: [float64] unitless (shape: (y:1, x:4096))]

As previously two datasets are returned due to the imcompatible types and shapes of the experiments.

If one desire to load of files into separate datasets, then set the **merge** attribute to False.

In [6]:
LZ1 = scp.read_opus("irdata/OPUS", merge=False)
LZ1

[NDDataset: [float64] a.u. (shape: (y:1, x:2567)),
 NDDataset: [float64] a.u. (shape: (y:1, x:2567)),
 NDDataset: [float64] a.u. (shape: (y:1, x:2567)),
 NDDataset: [float64] a.u. (shape: (y:1, x:2567)),
 NDDataset: [float64] unitless (shape: (y:1, x:4096))]

### Loading given type of OPUS spectra

By default absortion spectra (**AB**) are load, if present in the file.

Opus file however can contain several types of spectra and they can be retrieved eventually using the correct type in the call to `read_opus`:

Types possibly availables and readables by `read_opus` are listed here:

- `AB`: Absorbance (default if present in the file)
- `TR`: Transmittance
- `KM`: Kubelka-Munk
- `RAM`: Raman
- `EMI`: Emission
- `RFL`: Reflectance
- `LRF`: log(Reflectance)
- `ATR`: ATR
- `PAS`: Photoacoustic
- `RF`: Single-channel reference spectra
- `SM`: Single-channel sample spectra
- `IGRF`: Reference interferogram
- `IGSM`: Sample interferogram
- `PHRF`: Reference phase
- `PHSM`: Sample phase

It is possible to know which are the other types availables in the original file:

In [7]:
Z.meta.other_data_types

['RF', 'IGRF', 'AB', 'SM', 'PHSM', 'IGSM']

Thus if one wants to load the single-channel sample spectra, the read function syntax would be:

In [8]:
ZSM = scp.read_opus("irdata/OPUS/test.0002", type="SM")
ZSM

NDDataset: [float64] unitless (shape: (y:1, x:2567))

### Reading OPUS file Metadata

As just seen above, more informations can be obtained on the experiment and spectrometer parameters using the dataset metadata (**meta** attribute).

For instance, to get a display of all metadata:


In [9]:
Z.meta

To display only a sub-group of metadata, you can use :

In [10]:
Z.meta.params.fourier_transform

or a single parameter:

In [11]:
Z.meta.params.optical.bms

In [12]:
Z.meta.params.optical.bms.value

'KBr'

### Acting on a parameter

Metadata modification, addition, deletion are forbidden by default (`readonly=True`).

For instance, if one want to add a new parameters, the following is not permitted and then raise an error:

In [13]:
try:
    Z.meta.xxx = "forbidden"
except ValueError:
    scp.error_("meta data dictionary is read only!")

 ERROR | SpectroChemPyError: meta data dictionary is read only!


To add a new value (or to modifify/delete an existing value), the `readonly` flag must be unset before the operation:

In [14]:
Z.meta.readonly = False
Z.meta.xxx = "permitted"
Z.meta

It is advised to set back the `readonly` flag to True.